In [9]:
# linear algebra
import numpy as np 

# data processing
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
# data processing
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

#Dealing with Missing Values in Age variable
data = [train_df, test_df]
for dataset in data:
    mean = train_df["Age"].mean()
    std = test_df["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_df["Age"].astype(int)
train_df["Age"].isnull().sum()

#Categorizing Age variable into limited groups
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

#Converting gender to a numeric variable
genders = {"male": 0, "female": 1}
for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(0)

In [12]:
train_x = train_df[['Age','Sex','Pclass','Fare','SibSp','Parch']]
train_y = train_df[['Survived']]
test_x = test_df[['Age','Sex','Pclass','Fare','SibSp','Parch']]

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_x, train_y.values.ravel())
random_forest.score(train_x, train_y)
acc_random_forest = round(random_forest.score(train_x, train_y) * 100, 2)
acc_random_forest

94.95

In [13]:
titanic_survival_predictor(25,1,1,3,4,2)

[0.725]

In [7]:
import tabpy_client
from tabpy.tabpy_tools.client import Client

def titanic_survival_predictor( _arg1, _arg2, _arg3, _arg4,_arg5, _arg6):
    import pandas as pd

    #Get the new app's data in a dictionary
    row = {'Age': _arg1,
           'Sex': _arg2,
           'Pclass': _arg3,
           'Fare': _arg4,
           'SibSp': _arg5,
           'Parch': _arg6 }
    
    #Convert it into a dataframe
    test_data = pd.DataFrame(data = row,index=[0])

    predprob_survival = random_forest.predict_proba(test_data)
    return [probability[1] for probability in predprob_survival]

client = tabpy_client.Client('http://localhost:9004/')
client.deploy('titanic_survival_predictor', titanic_survival_predictor,'Predicts survival probability', override = True)